In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    confusion_matrix, 
    classification_report,
    roc_auc_score
)
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.multiclass import OneVsRestClassifier
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [2]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
DATA_ROOT = "../../data/images/images" 
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
VAL_DIR = os.path.join(DATA_ROOT, "validation") 
TEST_DIR = "../../data/images/asd"
EMOTION_LABELS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
BATCH_SIZE = 32
IMG_SIZE = 227 
NUM_WORKERS = 4

SAMPLE_BATCH_DIR = "saved_samples"
SAMPLE_BATCH_FILE = os.path.join(SAMPLE_BATCH_DIR, "sample_batch.pt")
SAMPLE_FEATURES_FILE = os.path.join(SAMPLE_BATCH_DIR, "sample_features.pt")
os.makedirs(SAMPLE_BATCH_DIR, exist_ok=True)


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [13]:
RF_N_ESTIMATORS = 100
RF_RANDOM_STATE = 42
RF_MAX_DEPTH = None
RF_MIN_SAMPLES_SPLIT = 2
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

In [14]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomRotation(degrees=90),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

In [15]:
class EmotionDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []  
        for label_idx, emotion in enumerate(EMOTION_LABELS):
            emotion_dir = os.path.join(data_dir, emotion)
            if os.path.exists(emotion_dir):
                for img_name in os.listdir(emotion_dir):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.images.append(os.path.join(emotion_dir, img_name))
                        self.labels.append(label_idx)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [16]:
try:
    train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)
    val_dataset = datasets.ImageFolder(root=VAL_DIR, transform=val_test_transform)
    test_dataset = datasets.ImageFolder(root=TEST_DIR, transform=val_test_transform)
    print(f"Jumlah data training: {len(train_dataset)}")
    print(f"Jumlah data validation: {len(val_dataset)}")
    print(f"Jumlah data test: {len(test_dataset)}")
    print(f"Kelas yang terdeteksi: {train_dataset.classes}")
    
except Exception as e:
    
    train_dataset = EmotionDataset(TRAIN_DIR, transform=train_transform)
    val_dataset = EmotionDataset(VAL_DIR, transform=val_test_transform)
    test_dataset = EmotionDataset(TEST_DIR, transform=val_test_transform)
    
    print(f"Jumlah data training: {len(train_dataset)}")
    print(f"Jumlah data validation: {len(val_dataset)}")
    print(f"Jumlah data test: {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

Jumlah data training: 63750
Jumlah data validation: 7066
Jumlah data test: 0


In [18]:
class ResNet50FeatureExtractor(nn.Module):
    def __init__(self):
        super(ResNet50FeatureExtractor, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)        
        self.resnet50.fc = nn.Identity()
        
        for param in self.resnet50.parameters():
            param.requires_grad = False
    
    def forward(self, x):
        features = self.resnet50(x)
        return features

In [19]:
feature_extractor = ResNet50FeatureExtractor().to(device)
feature_extractor.eval()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\MufliDevs/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:02<00:00, 39.4MB/s]


ResNet50FeatureExtractor(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequ

In [ ]:
with torch.no_grad():
    sample_batch, _ = next(iter(train_loader))
    sample_batch = sample_batch.to(device)
    sample_features = feature_extractor(sample_batch)
    print(f"Ukuran fitur yang diekstrak: {sample_features.shape}")
    print(f"Dimensi fitur per gambar: {sample_features.shape[1]}")


In [ ]:
def extract_features_and_labels(dataloader, feature_extractor, device):
    feature_extractor.eval()
    all_features = []
    all_labels = []
    
    print("Memulai ekstraksi fitur...")
    
    with torch.no_grad():
        for batch_idx, (data, labels) in enumerate(dataloader):
            data = data.to(device)
            features = feature_extractor(data)
            features_np = features.cpu().numpy()
            labels_np = labels.numpy()
            
            all_features.append(features_np)
            all_labels.append(labels_np)
            
            if (batch_idx + 1) % 50 == 0:
                print(f"Batch {batch_idx + 1}/{len(dataloader)} selesai")
    
    features_array = np.vstack(all_features)
    labels_array = np.concatenate(all_labels)
    
    print(f"Ekstraksi fitur selesai!")
    print(f"Ukuran features: {features_array.shape}")
    print(f"Ukuran labels: {labels_array.shape}")
    
    return features_array, labels_array

In [ ]:
print("=== EKSTRAKSI FITUR DATASET TRAINING ===")
X_train, y_train = extract_features_and_labels(train_loader, feature_extractor, device)

print("\n=== EKSTRAKSI FITUR DATASET VALIDATION ===")
X_val, y_val = extract_features_and_labels(val_loader, feature_extractor, device)


print(f"Training features: {X_train.shape}, labels: {y_train.shape}")
print(f"Validation features: {X_val.shape}, labels: {y_val.shape}")


In [ ]:
rf_classifier = RandomForestClassifier(
    n_estimators=RF_N_ESTIMATORS,
    random_state=RF_RANDOM_STATE,
    max_depth=RF_MAX_DEPTH,
    min_samples_split=RF_MIN_SAMPLES_SPLIT,
    n_jobs=-1,  
    verbose=1   
)

print(f"Parameter Random Forest:")
print(f"- n_estimators: {RF_N_ESTIMATORS}")
print(f"- random_state: {RF_RANDOM_STATE}")
print(f"- max_depth: {RF_MAX_DEPTH}")
print(f"- min_samples_split: {RF_MIN_SAMPLES_SPLIT}")

In [ ]:
rf_classifier.fit(X_train, y_train)

In [ ]:
val_predictions = rf_classifier.predict(X_val)
val_probabilities = rf_classifier.predict_proba(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
val_f1 = f1_score(y_val, val_predictions, average='weighted')

In [ ]:
val_accuracy = accuracy_score(y_val, val_predictions)
val_f1_weighted = f1_score(y_val, val_predictions, average='weighted')
val_f1_macro = f1_score(y_val, val_predictions, average='macro')

In [ ]:
print(f"\n=== CLASSIFICATION REPORT ===")
class_names = EMOTION_LABELS if hasattr(train_dataset, 'classes') else [f'Class_{i}' for i in range(len(EMOTION_LABELS))]
print(classification_report(y_val, val_predictions, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_val, val_predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            cbar_kws={'label': 'Count'})

plt.title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
y_val_binary = label_binarize(y_val, classes=range(len(class_names)))
n_classes = y_val_binary.shape[1]
try:
    roc_auc_ovr = roc_auc_score(y_val_binary, val_probabilities, 
                                multi_class='ovr', average='weighted')
    print(f"ROC AUC Score (One-vs-Rest, weighted): {roc_auc_ovr:.4f}")
    roc_auc_ovo = roc_auc_score(y_val, val_predictions, 
                                multi_class='ovo', average='weighted')
    print(f"ROC AUC Score (One-vs-One, weighted): {roc_auc_ovo:.4f}")
    
except Exception as e:
    print(f"Error menghitung ROC AUC: {e}")
    print("Kemungkinan karena ada kelas yang tidak muncul di test set")
    
    print("\nMenghitung ROC AUC per kelas:")
    for i in range(n_classes):
        if len(np.unique(y_val_binary[:, i])) > 1:  
            auc = roc_auc_score(y_val_binary[:, i], val_probabilities[:, i])
            print(f"{class_names[i]}: {auc:.4f}")